# <center>CMPBIO210, IB120, IB201
# <center>"Introduction to Quantitative Methods in Biology"
# <center>Lecture 5. ODE Model of Predator-Prey Dynamics
## <center>Denis Titov

Optional reading resources about Lotka-Volterra model:  
https://jmahaffy.sdsu.edu/courses/f09/math636/lectures/lotka/qualde2.html  
https://services.math.duke.edu/education/ccp/materials/diffeq/predprey/pred1.html

**The goal of this jupyter notebook is to discuss how to formulate, solve and analyze ODE model of predator-prey dynamics and how to find parameters that fit the data**

In [ ]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize, Bounds, differential_evolution, shgo,dual_annealing
import pandas as pd

## Lotka-Volterra ODE Model

Assumtions of the Lotka-Volterra model:
- $Hare$ eat unlimited supply of food and grow at rate $b \cdot Hair$ in absence of $Lynx$
- $Lynx$ population declines at a rate $d \cdot Lynx$ in absence of $Hare$
- $Lynx$ eat $Hare$ at rate $k_1 \cdot Lynx \cdot Hare$ and have unlimited appetite
- $Lynx$ population grows at rate $k_2 \cdot Hare \cdot Lynx$ in presence of $Hare$  

The model then is:  
  
$\frac{dHare}{dt}=b \cdot Hare - k_1 \cdot Lynx \cdot Hare$  
  
$\frac{dLynx}{dt}=k_2 \cdot Lynx \cdot Hare - d \cdot Lynx$  

Same equations can be used to described so called autocatalytic chemical reactions that are often shown in intro chemistry classes when one of the reactants has color so that reaction contineously chanign color.  
See examples video here: https://en.wikipedia.org/wiki/Briggs–Rauscher_reaction

Let's solve this model using solve_ivp()

In [ ]:
b = 1
k1 = 0.1
k2 = 0.1
d = 1

def LotkaVolterra(t, y):
    Hare, Lynx = y
    dHare_dt = (b - k1 * Lynx) * Hare
    dLynx_dt = (k2 * Hare - d) * Lynx
    return [dHare_dt, dLynx_dt]


initial_condition = [1, 1]
tspan = [0, 100]
soln = solve_ivp(
    LotkaVolterra, 
    tspan, 
    initial_condition) 

plt.figure(figsize=(8,5), dpi=100)
plt.plot(soln.t, soln.y[0],label="Hare")
plt.plot(soln.t, soln.y[1],label="Lynx")
plt.xlabel("Time, $t$", fontsize=20)
plt.ylabel("Population Size", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()
plt.legend(fontsize=15);

It's always good to check some basic assumptions of the model to make sure you didn't make simply coding or model generation mistake.  
Do sanity check of the model.  
For example:
- $Hare$ population should grow exponentially in absence of $Lynx$
- $Lynx$ population should decline exponentially in absence of $Hare$

## Phase plot of Lotka-Volterra ODE Model


Let's plot the solution of this model without solving ODEs.  
The idea is to sketch the dynamics of the system on a plot with $Hare$ and $Lynx$ values as x and y axis and arrows at a given [$Hare, Lynx$] coordinate that point in the direction of change of $Hare$ and $Lynx$ given by definition of derivative by the vector [$\frac{dHare}{dt}, \frac{dLynx}{dt}$].

In [ ]:
b = 1
k1 = 0.1
k2 = 0.1
d = 1

Hare, Lynx = np.mgrid[0:20:15j,0:20:15j]

dHare_dt, dLynx_dt = (b - k1 * Lynx) * Hare, (k2 * Hare - d) * Lynx
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14,7), dpi=100, constrained_layout=True)
axs[0].quiver(Lynx, Hare, dLynx_dt, dHare_dt)
axs[1].streamplot(Lynx, Hare, dLynx_dt, dHare_dt)

for ax in axs:
    ax.tick_params(labelsize=15)
    #ax.set_xlim(0,1)
    #ax.set_ylim(0,1)
    ax.grid(which='major', axis='both')
    ax.set_xlabel("Lynx",fontsize=20)
    ax.set_ylabel("Hair",fontsize=20)

## Identification of fixed points
Also referred to as equilibrium or steady state points
  
  Fixed points are points where $Hare$ and $Lynx$ population don't change/stay fixed. In ODE math language that would be:
  
  $\frac{dHare}{dt}=0$  
  
  $\frac{dLynx}{dt}=0$  
  
  We can find fixed points by solving the equations below:  
  
$\frac{dHare}{dt}=b \cdot Hare - k_1 \cdot Lynx \cdot Hare=0$  
  
$\frac{dLynx}{dt}=k_2 \cdot Lynx \cdot Hare - d \cdot Lynx=0$  


In [ ]:
Hare, Lynx, b, k1, k2, d = sym.symbols("Hare, Lynx, b, k1, k2, d")
sym.nonlinsolve(
    [(b - k1 * Lynx) * Hare, (k2 * Hare - d) * Lynx],
    [Hare,Lynx]
)

Confirm that these are fixed points by plugging them as initial conditions in solve_ivp().  
If these are initial point, you should observe straight horizontal line solutins because $\frac{dHare}{dt}=0$ and $\frac{dLynx}{dt}=0$. 

## Analysis of stability of fixed points
It is often interesting to know if fixed points are stable (i.e. system returns to fixed point after small perturbation) or unstable (i.e. system moves away from fixed points after small perturbation).  

You can check the stability of fixed points by change the initial values in solve_ivp() by 1% away from fixed point and seeing what happens.  

Try this using solve_ivp() code above.  
  
  It is also possible to analytically figure out whether fixed points are stable using eigenvalues and eigenvectors of matrix representation of Lotka-Volterra equations but we won't have time to review this during lectures.

## Fitting Lotka-Volterra model to data to find constants
Lets explore how we can measure the constants by fitting the Lotka-Volterra model to data

### Data for fitting

In [ ]:
LotkaVolterra_Data = pd.read_csv("Hare_Lynx_data.csv",index_col=None)

# Convert Pandas columns into numpy arrays for later calculations
Hares_Data=LotkaVolterra_Data['Hares (x1000)'].to_numpy()
Lynx_Data=LotkaVolterra_Data['Lynx (x1000)'].to_numpy()

LotkaVolterra_Data

Let's plot the data and compare to our model

In [ ]:
b = 1
k1 = 0.1
k2 = 0.1
d = 1

def LotkaVolterra(t, y):
    Hare, Lynx = y
    dHare_dt = (b - k1 * Lynx) * Hare
    dLynx_dt = (k2 * Hare - d) * Lynx
    return [dHare_dt, dLynx_dt]


initial_condition = [Hares_Data[0], Lynx_Data[0]]
tspan = [0, 100]
t_eval = np.linspace(0,20,21)
soln = solve_ivp(
    LotkaVolterra, 
    tspan, 
    initial_condition,
    t_eval = t_eval
) 

plt.figure(figsize=(8,5), dpi=100)
plt.plot(soln.t, soln.y[0], 'gx--', label="Hare Model")
plt.plot(np.linspace(0,20,21), Hares_Data, 'go-', label="Hare Data")
plt.plot(soln.t, soln.y[1], 'rx--',label="Lynx Model")
plt.plot(np.linspace(0,20,21), Lynx_Data, 'ro-',label="Lynx Data")
plt.xlabel("Time, $t$", fontsize=20)
plt.ylabel("Population Size", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()
plt.legend(fontsize=15, loc="upper right");

### Idea behind the method for fitting ODE model to data

Our goal is to determine the values of $b$, $k1$, $k2$ and $d$ so that our model is as close to data as possible.  
  
  There're many ways to do this and the whole scipy.optimize library is dedicated to various ways of perform this operation.  
  
  One simple way to do this is to calculate  
  
  $\overset{All \space Data \space Points}{ \underset{n=1}{\sum}} (Data(t)-Model(t))^2$  
  
  for every point of $Hare$ and $Lynx$ data and then change $b$, $k1$, $k2$ and $d$ to make this $\sum$ as small as possible.  
  
  Let's implement the above manually and then using SciPy functions.

In [ ]:
def fitting_fun(parameters):
    b, k1, k2, d = parameters
    
    def LotkaVolterra(t, y):
        Hare, Lynx = y
        dHare_dt = (b - k1 * Lynx) * Hare
        dLynx_dt = (k2 * Hare - d) * Lynx
        return [dHare_dt, dLynx_dt]
    
    #Choose initial_condition to match initial data
    initial_condition = [Hares_Data[0], Lynx_Data[0]]
    tspan = [0, 20]
    
    #Choose t_eval so ODE solution timepoint match data timepoints
    t_eval = np.linspace(0,20,21)
    soln = solve_ivp(
        LotkaVolterra, 
        tspan, 
        initial_condition,
        t_eval = t_eval
    )
    return (np.append(soln.y[0] - Hares_Data, soln.y[1] - Lynx_Data)**2).sum()

parameters=[1, 1, 1, 1]
fitting_fun(parameters)

### Local minimization

Let's use automatic minimization algorithm to find ODE parameters b, k1, k2, d that minimize fitting_fun().  
  
Minimization of function is required for a large number of quantitative methods and SciPy library has a large number of minimization algorirthms as part of scipy.optimize. Similar to how SciPy has many ODE solvers implemented in solve_ivp(), it has minimize() with many minimization algorithms.
  
Let's try minimize() function with boundary conditions.

In [ ]:
# bounds can be used to limit the range of parameters that minimize() searches over
# bounds will speedup minimize()
bounds = Bounds([0.001,0.001,0.001,0.001], [1,1,1,1])

# minimize() requires you to provide an initial guess of parameter values
initial_guess=[0.5, 0.05, 0.05, 0.5]

local_fit = minimize(fitting_fun, initial_guess, bounds=bounds)
local_fit

Let's check how good are the new parameters

In [ ]:
b, k1, k2, d = local_fit.x

def LotkaVolterra(t, y):
    Hare, Lynx = y
    dHare_dt = (b - k1 * Lynx) * Hare
    dLynx_dt = (k2 * Hare - d) * Lynx
    return [dHare_dt, dLynx_dt]


initial_condition = [30, 4]
tspan = [0, 100]
t_eval = np.linspace(0,20,1000)
soln = solve_ivp(
    LotkaVolterra, 
    tspan, 
    initial_condition,
    t_eval = t_eval
) 

plt.figure(figsize=(8,5), dpi=100)
plt.plot(soln.t, soln.y[0], 'g--', label="Hare Model")
plt.plot(np.linspace(0,20,21), Hares_Data, 'go-', label="Hare Data")
plt.plot(soln.t, soln.y[1], 'r--',label="Lynx Model")
plt.plot(np.linspace(0,20,21), Lynx_Data, 'ro-',label="Lynx Data")
plt.xlabel("Time, $t$", fontsize=20)
plt.ylabel("Population Size", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()
plt.legend(fontsize=15, loc="upper right");

Try changing the *initial_guess* and see how your results will change.  
  
  You should have noticed that minimize() provides the best answer only if *initial_guess* is somewhat close to best answer. This is becasue some functions can have global minimum and local minimum.  minimize() only find local minimum so if you start far away from best answer there's a chance that minimize() will be stuck in local minimum.  

### Global minimization
  Due to the reason described above, there're global minimization algorithms that are essentially algorithms geared to choose the best *initial_guess* conditions to ensure that minimize() doesn't get stuck in local minimum.  
  
  Lets' use one of global minimization algorithms called differential_evolution(). The idea behind this method is that you start with population and let it mutate and recombine with selective pressure of minimizing the function that you're interested in.  
  
  differential_evolution() finds our parameters without an *initial_guess*
  
  You can read about differential_evolution() here: https://doi.org/10.1023%2FA%3A1008202821328

In [ ]:
%%time
parameters_bounds = [(0.001, 1),(0.001, 1),(0.001, 1),(0.001, 1)]
global_fit = differential_evolution(fitting_fun, parameters_bounds)
global_fit

In [ ]:
b, k1, k2, d = global_fit.x

def LotkaVolterra(t, y):
    Hare, Lynx = y
    dHare_dt = (b - k1 * Lynx) * Hare
    dLynx_dt = (k2 * Hare - d) * Lynx
    return [dHare_dt, dLynx_dt]


initial_condition = [30, 4]
tspan = [0, 100]
t_eval = np.linspace(0,20,1000)
soln = solve_ivp(
    LotkaVolterra, 
    tspan, 
    initial_condition,
    t_eval = t_eval
) 

plt.figure(figsize=(8,5), dpi=100)
plt.plot(soln.t, soln.y[0], 'g--', label="Hare Model")
plt.plot(np.linspace(0,20,21), Hares_Data, 'go-', label="Hare Data")
plt.plot(soln.t, soln.y[1], 'r--',label="Lynx Model")
plt.plot(np.linspace(0,20,21), Lynx_Data, 'ro-',label="Lynx Data")
plt.xlabel("Time, $t$", fontsize=20)
plt.ylabel("Population Size", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()
plt.legend(fontsize=15, loc="upper right");

Try boundaries (0.001,10) for all parameters differential_evolution optimization and see what happens. Note that using larger boundaries takes ~7.5min on my computer. If you're lazy, the solution is [6.22460119, 0.32922744, 0.2923544 , 9.46871131]